# Chapter 5 - Part 3: Deploy UDF to Snowflake via Snowpark

Welcome to chapter 5 of our Snowflake Data Scientist training series.

In chapter 5 we will look at model deployment options. The code is structured into three parts:
- Part 1: We train a model and save its binary file to disk, called the "pickle file".
- Part 2: We deploy the pickle file to an Azure Function and call it via API Integration from Snowflake
- Part 3: We deploy the pickle file directly to Snowflake using SnowPark.

Happy coding!

## ATTENTION: Python 3.8 is needed to run this notebook.
It's the only Python version that Snowpark currently supports.

In [ ]:
## Make sure you have Python 3.8 installed on your system for this. Else the package won't install.
!pip install snowflake-snowpark-python

### 1.) Connecting to Snowflake

To connect to your Snowflake instance, make sure you have all requirements installed and your connection details ready.

In [ ]:
import json, sys
assert sys.version_info == (3, 8), "You really need to run Python 3.8 for this code."
import snowflake.snowpark as snp

In [ ]:
with open('state.json') as sdf:
    state_dict = json.load(sdf)    

session = snp.Session.builder.configs(state_dict["connection_parameters"]).create()
session.use_warehouse(state_dict['compute_parameters']['default_warehouse'])
return session, state_dict

### 2.) Start Working with simple example
As a first step we will work on a simple example. Let's multiply some integers and see how it all comes together.

The steps that need to be done:
- define the UDF in Python as you normally would
- register the UDF in Snowflake via Snowpark


In [2]:
def multiply_by_four(input_int_py: int):
  return input_int_py*4

Next up we register UDF in Snowflake. Also have a look at interworks code repository and their great examples: https://github.com/interworks/Snowflake-Python-Functionality/tree/main/Created%20via%20Snowpark/User%20Defined%20Functions.

Make sure you have an internal stage "UDF_STAGE" available in Snowflake.

In [ ]:
### Add packages and data types
from snowflake.snowpark.types import IntegerType

### Create the temp stage
session.sql('create stage if not exists UDF_STAGE').collect() 

### Upload UDF to Snowflake
session.udf.register(
    func = multiply_by_four
  , return_type = IntegerType()
  , input_types = [IntegerType()]
  , is_permanent = True
  , name = 'snp_multiply_by_four'
  , replace = True
  , stage_location = '@UDF_STAGE'
)

Finally lets do some testing, first via Snowpark, later via Snowflake UI directly.

In [ ]:
session.sql('''
  SELECT snp_multiply_by_four(20)
''').show()

Done! We now have a working UDF using Snowpark in Snowflake.

### 3.) Uploading our real UDF to Snowflake
Again we have to do two steps:
- define the UDF in Python as you normally would
- register the UDF in Snowflake via Snowpark

Our UDF in this case is just the inference function of our model. We load the pickle file and then call predict on it.

In [47]:
import joblib
import sys
from sklearn.ensemble import RandomForestRegressor
rfr = joblib.load('randomforest_classifier.joblib.pkl')

def predict_bp_after(sex_agrgrp_position, bp_before):       
    return rfr.predict([[sex_agrgrp_position, bp_before]])[0]

Next up we register UDF in Snowflake. Again make sure you have an internal stage "UDF_STAGE" available in Snowflake.

In [ ]:
### Add packages and data types
from snowflake.snowpark.types import StringType, IntegerType, FloatType
session.add_packages(['pandas', 'scikit-learn', 'joblib'])
session.add_import('randomforest_classifier.joblib.pkl')

### Create the temp stage
session.sql('create stage if not exists UDF_STAGE').collect() 

### Upload UDF to Snowflake
session.udf.register(
    func = predict_bp_after
  , return_type = FloatType()
  , input_types = [IntegerType(), IntegerType()]
  , is_permanent = True
  , name = 'snp_predict_bp_after'
  , replace = True
  , stage_location = '@UDF_STAGE'
)

Finally lets do some testing, first via Snowpark, later via Snowflake UI directly.

In [ ]:
session.sql('''
  SELECT snp_predict_bp_after(0, 155)
''').show()

### Done. Close it.

In [17]:
session.close()